**Team Name:** DataNerds

**Student Details:**

--------------------------

Name : Reshma Vijay Jawale

Student Id : A0236581B

--------------------------
Name : Aiden Low Yew Woei

Student Id : A0121969W

--------------------------
Name : Raivat Bhupesh Shah 

Student Id : A0184879A

--------------------------

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from IPython.display import display
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Cleaning methods
import library_code.imputation as imputation
import library_code.auxiliary as auxiliary
import library_code.cleaning as cleaning
import library_code.constants as constants

# Scikit learn
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import KNNImputer
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge

from scipy.spatial import Voronoi, voronoi_plot_2d
from matplotlib.patches import Rectangle, Circle
from matplotlib.colors import ListedColormap

from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

# For adaboost
from sklearn.ensemble import AdaBoostClassifier


import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

99


In [3]:
df_prices_train = pd.read_csv("../data/train.csv")
df_prices_train_demo = pd.read_csv("../data/train.csv")
df_prices_test = pd.read_csv("../data/test.csv")
df_commercial_centres = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-commerical-centres.csv")
df_train_stations = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-train-stations.csv")
df_shopping_malls = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-shopping-malls.csv")
df_primary_schools = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-primary-schools.csv")
df_secondary_schools = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-secondary-schools.csv")
print("Property Prices train dataset", df_prices_train.shape)
display(df_prices_train.head())
print("Aux: Commercial Centres", df_commercial_centres.shape)
display(df_commercial_centres.head())
print("Aux: Train Stations", df_train_stations.shape)
display(df_train_stations.head())
print("Aux: Shopping Malls", df_shopping_malls.shape)
display(df_shopping_malls.head())
print("Aux: Primary Schools", df_primary_schools.shape)
display(df_primary_schools.head())
print("Aux: Secondary Schools", df_secondary_schools.shape)
display(df_secondary_schools.head())

Property Prices train dataset (26048, 23)


,listing_id,name,street,type,model,market_segment,type_of_area,bedrooms,bathrooms,district,...,lat,lng,tenure,built_year,no_of_units,area_size,eco_category,accessibility,date_listed,price
0,6998418,seascape,57 cove way,condominium,condominium,ocr,strata,3,4.0,4,...,1.239337,103.837487,leasehold/99 years,2011.0,151.0,2336.0,uncategorized,guarded,2021-05-04,5390000.0
1,2046604,la maison,10 moulmein rise,apartment,apartment,ocr,strata,3,3.0,11,...,1.319533,103.847030,freehold,1999.0,24.0,1259.0,uncategorized,guarded,2021-05-07,2310000.0
2,7563513,viva,2 suffolk walk,condominium,condominium,ocr,strata,4,3.0,11,...,1.315656,103.844445,freehold,2012.0,235.0,1959.0,uncategorized,guarded,2021-06-30,5279500.0
3,3071445,urban treasures,205 jalan eunos,condominium,condominium,ocr,strata,3,2.0,14,...,1.329367,103.905791,freehold,NaN,237.0,883.0,uncategorized,guarded,2022-01-02,1843600.0
4,9667539,infini at east coast,east coast road,apartment,apartment,ocr,strata,3,3.0,15,...,1.309176,103.911352,freehold,NaN,36.0,1066.0,uncategorized,guarded,2021-12-24,2262700.0


Aux: Commercial Centres (38, 4)


,name,type,lat,lng
0,Central Business District,CR,1.286768,103.854529
1,Jurong Lake District,CR,1.334085,103.734513
2,Seletar Regional Centre,CR,1.409435,103.866820
3,Tampines Regional Centre,CR,1.353180,103.945188
4,Woodlands Regional Centre,CR,1.436748,103.786485


Aux: Train Stations (166, 6)


...

,name,lat,lng,wikipedia_link
0,100 AM,1.275012,103.841372,NaN
1,313@Somerset,1.301013,103.838540,NaN
2,Aperia,1.309714,103.863837,NaN
3,Balestier Hill Shopping Centre,1.325442,103.842521,NaN
4,Bugis Cube,1.298276,103.855555,NaN


Aux: Primary Schools (191, 3)


,name,lat,lng
0,Admiralty Primary School,1.442941,103.800345
1,Ahmad Ibrahim Primary School,1.433849,103.832710
2,Ai Tong School,1.360713,103.833018
3,Alexandra Primary School,1.291284,103.824066
4,Anchor Green Primary School,1.390380,103.887354


Aux: Secondary Schools (150, 3)


,name,lat,lng
0,Admiralty Secondary School,1.445912,103.802908
1,Ahmad Ibrahim Secondary School,1.436095,103.830055
2,Anderson Secondary School,1.375417,103.851723
3,Anglican High School,1.330218,103.941759
4,Anglo-Chinese School (Barker Road),1.319709,103.835661


In [4]:
# First, drop all fields in TO_IGNORE as these columns are either redundant
# e.g. we drop model because we have type
# or they only have one unique value, e.g. 'market_segment' and 'type_of_area'.
df_prices_train = df_prices_train.drop(columns=constants.TO_IGNORE)
df_prices_test = df_prices_test.drop(columns=constants.TO_IGNORE)

In [5]:
# Then, we clean up the tenure fields to 3 categories: 60 years, 99 years, and freehold/999
df_prices_train['tenure'] = df_prices_train['tenure'].apply(cleaning.tenure_to_binary)
df_prices_test['tenure'] = df_prices_test['tenure'].apply(cleaning.tenure_to_binary)
# Perform one hot encoding
df_prices_train = cleaning.categorical_to_onehot(df_prices_train, ['tenure', 'type', 'region', 'planning_area'])
df_prices_test = cleaning.categorical_to_onehot(df_prices_test, ['tenure', 'type', 'region', 'planning_area'])
# Remove 'planning_area_seletar' column in test as that's not seen in train.
df_prices_test.drop(columns=['planning_area_seletar'], inplace=True)

In [6]:
# Take 4+1 as 5
df_prices_train['bedrooms'] = df_prices_train['bedrooms'].apply(cleaning.process_bedroom_sum)
df_prices_test['bedrooms'] = df_prices_test['bedrooms'].apply(cleaning.process_bedroom_sum) 

# Take 4+1 as 4.5
# df_prices_train['bedrooms'] = df_prices_train['bedrooms'].apply(cleaning.process_bedroom_half)
# df_prices_test['bedrooms'] = df_prices_test['bedrooms'].apply(cleaning.process_bedroom_half) 
df_prices_train[df_prices_train['bedrooms'].isna()]

,bedrooms,bathrooms,district,lat,lng,no_of_units,area_size,price,tenure_60,tenure_99,...,planning_area_sembawang,planning_area_sengkang,planning_area_serangoon,planning_area_singapore river,planning_area_southern islands,planning_area_tampines,planning_area_tanglin,planning_area_toa payoh,planning_area_woodlands,planning_area_yishun
59,NaN,1.0,15,1.314620,103.932237,116.0,409.0,715000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,NaN,NaN,6,1.292988,103.851047,39.0,6609.0,27423000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
203,NaN,1.0,2,1.274644,103.844742,360.0,365.0,990000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
296,NaN,1.0,2,1.274644,103.844742,360.0,365.0,1100000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
443,NaN,NaN,22,1.335557,103.742417,738.0,506.0,1045000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25849,NaN,1.0,18,1.376176,103.960488,473.0,571.0,693000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25866,NaN,2.0,23,1.379727,103.760191,338.0,624.0,825000.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25884,NaN,1.0,1,1.277083,103.849181,510.0,441.0,1099800.0,0,1,...,0,0,0,0,0,0,0,0,0,0
25916,NaN,1.0,7,1.297510,103.856297,522.0,409.0,1098900.0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from geopy import distance
from tqdm import tqdm
tqdm.pandas()


def distance_ll(lat1: float, lng1: float, lat2: float, lng2: float) -> float:
    return distance.distance((lat1, lng1), (lat2, lng2))

def dist_to_nearest_item(expr: pd.DataFrame, aux_df: pd.DataFrame) -> str:
    return round(min(aux_df.apply(lambda x: distance_ll(expr['lat'], expr['lng'], x['lat'], x['lng']), axis=1)).km, 1)

# # Add distance to Primary School.
# df_prices_train['dist_primary_school'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)
# df_prices_test['dist_primary_school'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)


In [8]:
# Import auxiliary data
df_commercial_centres = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-commerical-centres.csv")
df_train_stations = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-train-stations.csv")
df_shopping_malls = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-shopping-malls.csv")
df_primary_schools = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-primary-schools.csv")
df_secondary_schools = pd.read_csv("../data/auxiliary-data/auxiliary-data/sg-secondary-schools.csv")

# Have a peek at the data
print("Aux: Commercial Centres", df_commercial_centres.shape)
display(df_commercial_centres.head())
print("Aux: Train Stations", df_train_stations.shape)
display(df_train_stations.head())
print("Aux: Shopping Malls", df_shopping_malls.shape)
display(df_shopping_malls.head())
print("Aux: Primary Schools", df_primary_schools.shape)
display(df_primary_schools.head())
print("Aux: Secondary Schools", df_secondary_schools.shape)
display(df_secondary_schools.head())

Aux: Commercial Centres (38, 4)


,name,type,lat,lng
0,Central Business District,CR,1.286768,103.854529
1,Jurong Lake District,CR,1.334085,103.734513
2,Seletar Regional Centre,CR,1.409435,103.866820
3,Tampines Regional Centre,CR,1.353180,103.945188
4,Woodlands Regional Centre,CR,1.436748,103.786485


Aux: Train Stations (166, 6)


,name,codes,lat,lng,opening_year,type
0,Eunos,EW7,1.319778,103.903252,1989.0,mrt
1,Chinese Garden,EW25,1.342352,103.732596,1988.0,mrt
2,Khatib,NS14,1.417383,103.832980,1988.0,mrt
3,Kranji,NS7,1.425177,103.762165,1996.0,mrt
4,Redhill,EW18,1.289562,103.816816,1988.0,mrt


Aux: Shopping Malls (174, 4)


,name,lat,lng,wikipedia_link
0,100 AM,1.275012,103.841372,NaN
1,313@Somerset,1.301013,103.838540,NaN
2,Aperia,1.309714,103.863837,NaN
3,Balestier Hill Shopping Centre,1.325442,103.842521,NaN
4,Bugis Cube,1.298276,103.855555,NaN


Aux: Primary Schools (191, 3)


,name,lat,lng
0,Admiralty Primary School,1.442941,103.800345
1,Ahmad Ibrahim Primary School,1.433849,103.832710
2,Ai Tong School,1.360713,103.833018
3,Alexandra Primary School,1.291284,103.824066
4,Anchor Green Primary School,1.390380,103.887354


Aux: Secondary Schools (150, 3)


,name,lat,lng
0,Admiralty Secondary School,1.445912,103.802908
1,Ahmad Ibrahim Secondary School,1.436095,103.830055
2,Anderson Secondary School,1.375417,103.851723
3,Anglican High School,1.330218,103.941759
4,Anglo-Chinese School (Barker Road),1.319709,103.835661


In [9]:
# Add distance to Primary School.
df_prices_train['dist_primary_school'] = df_prices_train.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)
df_prices_test['dist_primary_school'] = df_prices_test.progress_apply(lambda x: dist_to_nearest_item(x, df_primary_schools), axis=1)

100%|██████████| 7500/7500 [04:11<00:00, 29.84it/s]


## Adaboost Model

In [10]:
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train['price'].values.ravel()

model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__base_estimator__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
    'model__learning_rate': [0.1, 1.0],
}
# Repeats set at 3 to reduce model 
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, cv=cv, n_jobs=-1, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 6
n_possible_iterations: 5
min_resources_: 107
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 500
n_resources: 107
Fitting 30 folds for each of 500 candidates, totalling 15000 fits
----------
iter: 1
n_candidates: 167
n_resources: 321
Fitting 30 folds for each of 167 candidates, totalling 5010 fits
----------
iter: 2
n_candidates: 56
n_resources: 963
Fitting 30 folds for each of 56 candidates, totalling 1680 fits
----------
iter: 3
n_candidates: 19
n_resources: 2889
Fitting 30 folds for each of 19 candidates, totalling 570 fits
----------
iter: 4
n_candidates: 7
n_resources: 8667
Fitting 30 folds for each of 7 candidates, totalling 210 fits


In [18]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1903363.225523 using {'imputer__n_neighbors': 15}


In [16]:
clf = grid_results
X_test = df_prices_test[:]
y_pred = clf.predict(X_test)

In [ ]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("../predictions/submission_adaboost_with_pschool_tenure_oridinal.csv")

## Bagging Regressor

In [ ]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_jobs=-1)
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__max_features': [0.5, 0.7, 0.9, 1],
    'model__base_estimator__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

In [ ]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [ ]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_bagging_with_pschool_tenure_ordinal.csv")

## Bayesian Regressor

In [19]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = BayesianRidge()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 8682
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 5
n_resources: 8682
Fitting 10 folds for each of 5 candidates, totalling 50 fits
----------
iter: 1
n_candidates: 2
n_resources: 26046
Fitting 10 folds for each of 2 candidates, totalling 20 fits


In [20]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1903131.573506 using {'imputer__n_neighbors': 15}


In [21]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [22]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_bayesian_regression_with_pschool_tenure_ordinal.csv")

## Gradient Boosting Regression

In [23]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = GradientBoostingRegressor()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__subsample': [0.2, 0.5, 0.9, 1],
    'model__max_features': ['auto', 'sqrt'],
    'model__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

(26048, 56)
n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 35
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 2000
n_resources: 35
Fitting 10 folds for each of 2000 candidates, totalling 20000 fits
----------
iter: 1
n_candidates: 667
n_resources: 105
Fitting 10 folds for each of 667 candidates, totalling 6670 fits
----------
iter: 2
n_candidates: 223
n_resources: 315
Fitting 10 folds for each of 223 candidates, totalling 2230 fits
----------
iter: 3
n_candidates: 75
n_resources: 945
Fitting 10 folds for each of 75 candidates, totalling 750 fits
----------
iter: 4
n_candidates: 25
n_resources: 2835
Fitting 10 folds for each of 25 candidates, totalling 250 fits
----------
iter: 5
n_candidates: 9
n_resources: 8505
Fitting 10 folds for each of 9 candidates, totalling 90 fits
----------
iter: 6
n_candidates: 3
n_resources: 25515
Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [24]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 724506.447637 using {'imputer__n_neighbors': 5, 'model__max_depth': 5, 'model__max_features': 'auto', 'model__n_estimators': 500, 'model__subsample': 0.5}


In [25]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [26]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_gradient_boosting_with_pschool_tenure_oridinal.csv")

## KNeighbors Regression

In [ ]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = KNeighborsRegressor()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__n_neighbors': [i for i in range(1, 32, 2)]
}

cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

In [ ]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [ ]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_kneighbors_no_aux_tenure_oridinal.csv")

## Linear Regression

In [ ]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()
print(X_train.shape)
model = LinearRegression()
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__fit_intercept': [False, True],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

In [ ]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param)

In [ ]:
# Predict
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [ ]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("../predictions/submission_linear_regression_with_pschool_tenure_ordinal.csv")

## Random Forest

In [27]:
# Get data
X_train = df_prices_train.loc[:, df_prices_train.columns != 'price']
y_train = df_prices_train[['price']].values.ravel()

In [28]:
model = RandomForestRegressor(n_jobs=-1)
# Set up pipeline with imputer for proper grid search
# Reference: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html
pipeline = Pipeline(steps=[('imputer', KNNImputer()), ('model', model)])

# define grid search for hyperparameters
grid = {
    'imputer__n_neighbors': [3, 5, 7, 11, 15],
    'model__max_features': ['auto', 'sqrt'],
    'model__max_depth': [i for i in range(1, 21, 2)],
    'model__n_estimators': [25, 50, 100, 150, 250, 500],
}


cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# define grid search for hyperparameters
num_features = X_train.shape[1]

mse = make_scorer(mean_squared_error, greater_is_better=False)
grid_search = HalvingGridSearchCV(estimator=pipeline, param_grid=grid, n_jobs=-1, cv=cv, scoring=mse, verbose=1)
# Execute the grid search
grid_results = grid_search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 6
n_possible_iterations: 5
min_resources_: 107
max_resources_: 26048
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 600
n_resources: 107
Fitting 10 folds for each of 600 candidates, totalling 6000 fits
----------
iter: 1
n_candidates: 200
n_resources: 321
Fitting 10 folds for each of 200 candidates, totalling 2000 fits
----------
iter: 2
n_candidates: 67
n_resources: 963
Fitting 10 folds for each of 67 candidates, totalling 670 fits
----------
iter: 3
n_candidates: 23
n_resources: 2889
Fitting 10 folds for each of 23 candidates, totalling 230 fits
----------
iter: 4
n_candidates: 8
n_resources: 8667
Fitting 10 folds for each of 8 candidates, totalling 80 fits


In [29]:
# Get RMSE of grid results best score
best_rmse = (-grid_results.best_score_) ** 0.5
print("Best: %f using %s" % (best_rmse, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 939868.906608 using {'imputer__n_neighbors': 7, 'model__max_depth': 19, 'model__max_features': 'auto', 'model__n_estimators': 100}


In [30]:
clf = grid_results
X_test = df_prices_test.loc[:, df_prices_test.columns != 'price']
y_pred = clf.predict(X_test)

In [ ]:
result = pd.DataFrame({"Predicted": y_pred})
result.index.name = "Id"
result.to_csv("submission_forest_with_pschool_tenure_ordinal.csv")